# Allowed libraries
- Tensorflow (compatible with 1.12.x)
- Numpy
- Sklearn
- nltk
- Maplotlib
- gensim
- All the standard libraries
 

https://medium.com/the-artificial-impostor/nlp-four-ways-to-tokenize-chinese-documents-f349eb6ba3c3

https://stanfordnlp.github.io/CoreNLP/download.html

In [1]:
import os
import numpy as np
from typing import Tuple, List, Dict

import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.preprocessing.sequence import pad_sequences, TimeseriesGenerator

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#tf.__version__ fuck

In [3]:
def ChooseDataset(set_type):
    '''returns paths to Label and Input file for a specific dataset
    args: set_type
    return: Label_file, Input_file
    '''
    datasets = {"training":'../icwb2-data/training',
                "dev":'../icwb2-data/gold',
                "testing":'../icwb2-data/testing'}
        
    def get_file_names(path, type_='LabelFile'):
        x = []
        dev = True if path.split("/")[-1] == 'gold' else False #checks for dev
        for i in os.listdir(path):
            if dev and i.split("_")[1][:4]=='test': #eliminates 'training' from 'gold' 
                if os.path.splitext(i)[0].split("_")[-1] == type_:
                    x.append(os.path.join(path, i))
            elif not dev:
                if os.path.splitext(i)[0].split("_")[-1] == type_:
                    x.append(os.path.join(path, i))
        return x
    
    Label_files = get_file_names(path = datasets[set_type], type_ = 'LabelFile')
    Input_files = get_file_names(path = datasets[set_type], type_ = 'InputFile')
    names = ['msr','cityu','as','pku']
    choose = lambda i: i.split(".utf8")[0].split('/')[-1].split("_")[0]
    e, r = False, False
    chosen = False
    while not chosen:
        x = input("Choose from the following: {}".format(names))
        for i in range(len(Label_files)):
            if choose(Input_files[i]) == x: 
                Input_file = Input_files[i]
                e = True
            if choose(Label_files[i]) == x:
                Label_file = Label_files[i]
                r = True
            if e and r:
                chosen = True
            
    return Label_file, Input_file

In [43]:
class CreateDataset(object):
    '''makes feed files of combined unigrams and bigrams'''
    def __init__(self, LabelFile_path, InputFile_path, PaddingSize, set_type, TrainingVocab):
        self.Label_File = LabelFile_path
        self.Input_File = InputFile_path
        self.PaddingSize = PaddingSize
        self.set_type = set_type
        self.TrainingVocab = TrainingVocab
    
    def DateGen(self):
        '''creates labels from the label file'''
        
        features_vectors, word_to_index = self.FeatureGenerator() 
        labels = self.BIESToNumerical()
        #Optimal_Line_Length = int(np.mean([len(i) for i in features_vectors])) #length of longest line
        Optimal_Line_Length = self.PaddingSize
        
        #print("MAXLEN: {}".format(Optimal_Line_Length)) 
        padded_labels = pad_sequences(labels, truncating='pre', padding='post', maxlen = Optimal_Line_Length)
        
        y =  K.utils.to_categorical(padded_labels, num_classes=4)
        X = pad_sequences(features_vectors, truncating='pre', padding='post', maxlen = Optimal_Line_Length)
        
        info = {"MAXLEN": Optimal_Line_Length,
                "VocabSize": len(word_to_index)}
        return X, y, info, word_to_index
    
    def BIESToNumerical(self):
        '''Converts Label File from BIES encoding to numerical classes'''
        BIES = {'B' : 0, 'I' : 1, 'E' : 2, 'S' : 3}
        #numerical BIES class given to a line 
        labels = []
        with open(self.Label_File, 'r', encoding ='utf8') as f1:
            count = 0
            for line in f1:
                l = line.rstrip()
                labels.append([BIES[i] for i in l])
        return labels
    
    def FeatureGenerator(self):
        '''Generates features based on '''
        features_vectors = []
        if self.set_type == 'training':
            word_to_index = self.generateVocab()
        else:
            word_to_index = self.TrainingVocab

        with open(self.Input_File, 'r', encoding ='utf8') as f1:
            for line in f1:
                l = line.rstrip()
                grams = self.split_into_grams(l, 'uni_grams') + self.split_into_grams(l,'bi_grams')
                #difference is creating by grams line by line
#                 c=0
#                 if not self.set_type =='training':
#                     for q in grams:
#                         print(q, word_to_index_training.get(q))
#                         break
#                         if word_to_index_training.get(q) is None:
#                             q = '<UNK>'
#                             c+=1
#                         features_vectors.append(word_to_index[q])
#                 else:
                features_vectors.append([word_to_index.get(i, 0) for i in grams])
            #print("unknown words: {}\total: {}".format(c, len(word_to_index)))
        return features_vectors, word_to_index
    
    def generateVocab(self):
        '''
        Generates vocabulary based on file
        args: Inputfile, returns: word_to_index dict
        '''
        big_line = ''
        with open(self.Input_File, 'r', encoding ='utf8') as f1:
            for line in f1:
                big_line+=line.rstrip()
        final = self.split_into_grams(big_line, type_ = 'bi_grams') + self.split_into_grams(big_line, type_ = 'uni_grams')
        vocab = set(final)
        word_to_index = dict()
        word_to_index['<UNK>'] = 0
        word_to_index.update({value:key+1 for key,value in enumerate(vocab)})
        
        
        return word_to_index
        
    
    @staticmethod
    def split_into_grams(sentence: str, type_ = 'uni_grams') -> List[str]:
        """
        :param sentence Sentence as str
        :type_: uni_grams or _bigrams
        :return bigrams List of unigrams or bigrams
        """
        n = 1 if type_ == 'uni_grams' else 2
        grams = []
        for i in range(len(sentence)-1):
            gram = sentence[i:i+n]
            grams.append(gram)
        return grams

    

# training

In [44]:
type_ = "training"
Label_file, Input_file = ChooseDataset(type_)
A = CreateDataset(Label_file, Input_file, 10, type_, None)
X_train, y_train, info_train, word_to_index_training = A.DateGen()
print("X shape: {}\ny shape: {}".format(X_train.shape, y_train.shape))
print(info_train)

Choose from the following: ['msr', 'cityu', 'as', 'pku']pku
X shape: (19056, 10)
y shape: (19056, 10, 4)
{'MAXLEN': 10, 'VocabSize': 285201}


In [45]:
type_ = 'dev'
Label_file, Input_file = ChooseDataset(type_)
A = CreateDataset(Label_file, Input_file, 10, type_, word_to_index_training)
X_dev, y_dev, info_dev, _ = A.DateGen()
print("X shape: {}\ny shape: {}".format(X_dev.shape, y_dev.shape))
print(info_dev)

Choose from the following: ['msr', 'cityu', 'as', 'pku']pku
X shape: (1945, 10)
y shape: (1945, 10, 4)
{'MAXLEN': 10, 'VocabSize': 285201}


# model

In [47]:
#DEFINE SOME COSTANTS
MAX_LENGTH = 88
VOCAB_SIZE = info_dev['VocabSize']
EMBEDDING_SIZE = 32
HIDDEN_SIZE = 256
TO_BE_FOUND = info_dev['MAXLEN']

https://github.com/keras-team/keras/issues/1029 

Explains Timedistributed in many-to-many models

In [48]:
def create_keras_model(vocab_size, embedding_size, hidden_size, TO_BE_FOUND):
    print("Creating KERAS model")
    model = K.models.Sequential()
    model.add(K.layers.Embedding(vocab_size, embedding_size, mask_zero=True, input_length = TO_BE_FOUND))
    model.add(K.layers.LSTM(hidden_size, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    model.add(K.layers.TimeDistributed(K.layers.Dense(4, activation='softmax')))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

    return model

In [49]:
batch_size = 32
epochs = 10
model = create_keras_model(VOCAB_SIZE, EMBEDDING_SIZE, HIDDEN_SIZE, TO_BE_FOUND)
# Let's print a summary of the model
model.summary()

Creating KERAS model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 32)            9126432   
_________________________________________________________________
lstm (LSTM)                  (None, 10, 256)           295936    
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 4)             1028      
Total params: 9,423,396
Trainable params: 9,423,396
Non-trainable params: 0
_________________________________________________________________


In [50]:
cbk = K.callbacks.TensorBoard("logging/keras_model")
print("\nStarting training...")


Starting training...


In [51]:
# percent = 10
# size = int(len(X)/(1-percent))
# X_train = X[:2000]
# y_train = y[:2000]
# dev_x = X[-300:]
# dev_y = y[-300:]

# X_dev, y_dev
# X_train, y_train

In [52]:
print("X train {}".format(X_train.shape))
print("y train {}".format(y_train.shape))
print("X dev {}".format(X_dev.shape))
print("y dev {}".format(y_dev.shape))

X train (19056, 10)
y train (19056, 10, 4)
X dev (1945, 10)
y dev (1945, 10, 4)


In [53]:
max([max(i) for i in X_train]), max([max(i) for i in X_dev])

(285196, 285180)

In [54]:
min([min(i) for i in X_train]), min([min(i) for i in X_dev])

(0, 0)

In [55]:
X_train[:,0]

array([118066, 280596, 177867, ...,  66023, 243176, 174750], dtype=int32)

In [56]:
X_dev[:,0]

array([ 94686,  10200,  92011, ..., 207715, 185716,      0], dtype=int32)

In [ ]:
K.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

In [57]:

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
          shuffle=True, validation_data=(X_dev, y_dev), callbacks=[cbk]) 
print("Training complete.\n")



Train on 19056 samples, validate on 1945 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
19056/19056 [==============================] - 92s 5ms/sample - loss: 1.0432 - acc: 0.5760 - val_loss: 1.0082 - val_acc: 0.5751
Epoch 2/10
19056/19056 [==============================] - 106s 6ms/sample - loss: 0.8839 - acc: 0.6363 - val_loss: 1.0065 - val_acc: 0.5902
Epoch 3/10
19056/19056 [==============================] - 100s 5ms/sample - loss: 0.8068 - acc: 0.6698 - val_loss: 0.9868 - val_acc: 0.5984
Epoch 4/10
19056/19056 [==============================] - 98s 5ms/sample - loss: 0.7347 - acc: 0.7001 - val_loss: 1.0187 - val_acc: 0.5927
Epoch 5/10
19056/19056 [==============================] - 92s 5ms/sample - loss: 0.6665 - acc: 0.7289 - val_loss: 1.0202 - val_acc: 0.5935
Epoch 6/10
19056/19056 [==============================] - 94s 5ms/sample - loss: 0.6000 - acc: 0.7600 - val_loss: 1.1607 - val_acc: 0.5871
Epoch 7/10
19056/19056 [==============================] - 94s 5ms/s

In [65]:
Input_file

'../icwb2-data/testing/pku_test_simplified_InputFile.utf8'

In [66]:
type_ = 'testing'
Label_file, Input_file = ChooseDataset(type_)
A = CreateDataset(Label_file, Input_file, 10, type_, word_to_index_training)
X_test, y_test, info_test, _ = A.DateGen()
print("X shape: {}\ny shape: {}".format(X_test.shape, y_test.shape))
print(info_dev)

Choose from the following: ['msr', 'cityu', 'as', 'pku']pku
X shape: (1945, 10)
y shape: (1945, 10, 4)
{'MAXLEN': 10, 'VocabSize': 285201}


In [67]:
print("\nEvaluating test...")
loss_acc = model.evaluate(X_test, y_test, verbose=3)
print("Test data: loss = %0.6f  accuracy = %0.2f%% " % (loss_acc[0], loss_acc[1]*100))


Evaluating test...
Test data: loss = 5.877808  accuracy = 3.18% 


In [ ]:
!tensorboard --logdir logging

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
W0406 23:24:06.817950 123145545973760 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0406 23:24:06.835707 123145545973760 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well as one or more graph events.  Overwriting the graph with the newest event.
W0406 23:24:06.858180 123145545973760 plugin_event_accumulator.py:294] Found more than one graph event per run, or there was a metagraph containing a graph_def, as well 

- original file $\rightarrow$ simplified Chinese
- Input file $\rightarrow$ used to feed Bi-LSTM model
- Label file $\rightarrow$ used to test the predictions

TO DO: probably need a decoder

In [ ]:
1448/60